In [ ]:
# 한글폰트 사용 in colab
%matplotlib inline  

import matplotlib as mpl 
import matplotlib.pyplot as plt 
import matplotlib.font_manager as fm  

!apt-get update -qq
!apt-get install fonts-nanum* -qq

path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf' 
font_name = fm.FontProperties(fname=path, size=10).get_name()
print(font_name)
plt.rc('font', family=font_name)

fm._rebuild()
mpl.rcParams['axes.unicode_minus'] = False

교수님 위에 코드 실행시키시면 오류가 뜰 수도 있습니다.  
# 오류가 생기던 생기지 않던 **런타임 -> 런타임 다시 시작을 최소 1번 실행** 시켜주셔야 그래프에서 한글이 안깨집니다.  
감사합니다.!  
참고링크 : https://jonsyou.tistory.com/22

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import scipy.stats as sp
import scipy
plt.style.use('fivethirtyeight')

In [ ]:
dataset1 = pd.read_csv("/content/drive/MyDrive/datasets/데이터분석과제/전라남도수료기록.csv",encoding='CP949')
dt1_class = pd.read_csv('/content/drive/MyDrive/datasets/데이터분석과제/인재개발원 분류.csv',encoding='CP949')
dataset2 = pd.read_csv("/content/drive/MyDrive/datasets/데이터분석과제/행정안전부수료기록.csv",encoding='CP949')
dt2_class = pd.read_csv('/content/drive/MyDrive/datasets/데이터분석과제/행정안전부 분류.csv',encoding='CP949')

# 1.  1. 직무공통, 사회복지, 청렴 등 분야별 이수 결과 분석
  - 분류별 선호과목 조사 (수료자 수 조사)
  - 소분류별 수료율 조사(수료율 조사)

##분류별 선호과목 조사 (수료자 수 조사)

###전라남도 수료기록 분석

In [ ]:
dataset1.head()

In [ ]:
dataset1.info()

데이터셋을 '과정명'변수를 기준으로 결합하기 위해서 변수를 전처리했습니다. 

In [ ]:
l1 = []
for i in dataset1.과정명.str.split(']'):
  if len(i) == 2:
    l1.append(i[1].strip())
  elif len(i) == 1:
    l1.append(i[0].strip())
dataset1.과정명 = l1

l2=[]
for i in dt1_class.과정명.str.split(']'):
  if len(i) == 2:
    l2.append(i[1].strip())
  elif len(i) == 1:
    l2.append(i[0].strip())
dt1_class.과정명 = l2

In [ ]:
#1.1.1
print('전라남도 수료기록 과정명 변수에 있는 고유한 과정의 수 :',len(dataset1.과정명.unique()))
print('인재개발원 분류 과정명 변수에 있는 값의 수 : ',len(dt1_class.과정명.unique()))

In [ ]:
#1.1.2
dt1_class

결과:전라남도 수료기록을 인재개발원 분류 데이터셋과 과정명을 기준으로 결합하여 어떤 과목을 선호하는지 파악하고자 했습니다.  
그러나 다음과 같은 이유로 파악할 수 없었습니다.  
1.전라남도 수료기록에서 존재하는 모든 과정명에 대한 정확한 분류가 제시되어 있지 않습니다.고유한 값을 확인해 봤더니 전라남도 수료기록에 있는 과정명의 수가 인재개발원 분류에 있는 과정명의 수보다 많이 적습니다.  
2.인재개발원 분류 데이터가 대분류 중분류 소분류가 모두 같습니다.분류별 과목선호도를 파악할 수 없습니다.  

### 행정안전부 수료기록 분석

#### 데이터 전처리

In [ ]:
dataset2.info()

In [ ]:
l1 = []
"""
for i in dataset2.과정명.str.split(']'):
  length.append(len(i))
length=pd.Series(length)
length.value_counts()
"""
for i in dataset2.과정명.str.split(']'):
  if len(i) == 1:
    l1.append(i[0].strip())
  elif len(i) == 2:
    l1.append(i[1].strip())
    #print(i)
  elif len(i) == 3:
    l1.append(i[2].strip())
dataset2.과정명 = l1

In [ ]:
l2=[]
for i in dt2_class.과정명.str.split(']'):
  if len(i) == 1:
    l2.append(i[0].strip())
  elif len(i) == 2:
    l2.append(i[1].strip())
  elif len(i) == 3:
    l2.append(i[2].strip())
dt2_class.과정명 = l2

In [ ]:
print('행정안전부 수료기록 과정명 변수에 있는 고유한 과정의 수 :',len(dataset2.과정명.unique()))
print('행정안전부 분류 과정명 변수에 있는 값의 수 : ',len(dt2_class.과정명.unique()))

행정안전부 수료기록에 있는 각각의 관측치의 과정명에 대한 분류가 대부분 제시되어 있어 left join이 가능했습니다.

In [ ]:
행정안전부=pd.merge(dataset2,dt2_class,how='left',left_on = '과정명',right_on='과정명')[['대분류','중분류','소분류','과정명','학습기간','상시학습인정시간','소속','직급','성명','성별','수료여부']]
행정안전부[pd.isna(행정안전부.대분류)==True].과정명.unique()

In [ ]:
행정안전부.info()

In [ ]:
행정안전부.head()

위와 같은 과정명을 가진 관측치들은 left join후 정확한 대분류,중분류,소분류 널값었습니다.  
행정안전부 측에서 제공한 데이터셋에 위에 제시된 과정명에 대한 분류가 제시되지 않았기 때문입니다.  
이 경우 대,중,소분류가 Null값이 되어 분석을 위해 처리해야 합니다.  
저는 다음과 같은 처리를 하였습니다.
1. (집합연계),(행안부 대상),(나라콘텐츠)와 같은 괄호안의 키워드가 같은 자료들의 최빈값을 파악하여 대분류,중분류,소분류의 널값을 채워넣었습니다.
2. 키워드가 있으나 최빈값을 구할 수 없는 경우는 삭제했습니다.(해당 키워드를 포함하며 분류가 되어있는 다른 관측치가 없기 때문입니다.)
3. 키워드가 없는 관측치의 경우 삭제했습니다.



In [ ]:
a = {'집합연계':[],'행안부 대상':[],'나라콘텐츠':[],'MOOC':[],'마이크로러닝':[],'모바일':[],'자치인재원':[]}
for i in a.keys():
  print(i)
  for j in range(len(행정안전부)):
    if i in 행정안전부.과정명[j]:
      a[i].append(j)

In [ ]:
b = {'대분류':[],'중분류':[],'소분류':[]}
for i in b:
  for j in a.keys():
    if len(행정안전부[i][a[j]].value_counts().index) == 0:
      #대분류,중분류,소분류를 구할 수 없는 키워드는 값을 0으로
      b[i].append(0)
    else :
      b[i].append(행정안전부[i][a[j]].value_counts().index[0])

In [ ]:
b

In [ ]:
idx = {'집합연계':[],'행안부 대상':[],'나라콘텐츠':[],'MOOC':[],'마이크로러닝':[],'모바일':[],'자치인재원':[]}

for i in 행정안전부.loc[pd.isna(행정안전부.대분류)==True,:].index:
    for j in idx.keys():
      if j in 행정안전부.loc[i,'과정명']:
        idx[j].append(i)
        #print(i,j)

In [ ]:
행정안전부.loc[idx['집합연계'],:]

In [ ]:
h=0
for i in idx.values():
  for j in ['대분류','중분류','소분류']:
    행정안전부.loc[i,j] = b[j][h]
  h+=1

In [ ]:
print('키워드가 있으나 최빈값을 구할 수 없는 관측치의 수 : ',len(행정안전부.loc[idx['모바일'],:]))
print('키워드가 없는 관측치의 수 :',len(행정안전부[pd.isna(행정안전부.대분류)==True]))


In [ ]:
행정안전부=행정안전부.drop(index=행정안전부.loc[idx['모바일'],:].index)
행정안전부=행정안전부.drop(index=행정안전부[pd.isna(행정안전부.대분류)==True].index)

In [ ]:
행정안전부=행정안전부.reset_index(drop=True)
행정안전부

In [ ]:
행정안전부.info()

결과적으로 , 키워드값을 기준으로 대분류,중분류,소분류를 채운 널값이 없는 행정안전부 수료기록 데이터를 완성했습니다.  
대분류~수료여부까지 총 11개의 columns이 있습니다.  
관측치는 총 227796개 입니다.


In [ ]:
행정안전부

#### 분류별 선호과목 분석 (수료자 수 분석)

In [ ]:
plt.figure(figsize=(15,10))
plt.title('행정안전부 수료기록 데이터의 중분류별 빈도수')
a=행정안전부.중분류.value_counts()
ax=sns.barplot(x=a.index,y=a)
ax.set_xlabel("빈도수")

In [ ]:
#행정안전부 전체 데이터에 대한 과목별 빈도수 
#1.1.4
percent_5=int(len(행정안전부.과정명.value_counts())*0.05)
plt.figure(figsize=(30,20))
plt.title('행정안전부 수료기록 데이터의 빈도수 상위5% 과목들')
a=행정안전부.과정명.value_counts()[:percent_5]
ax=sns.barplot(y=a[:percent_5].index,x=a[:percent_5])
ax.set_xlabel("빈도수")

In [ ]:
#중분류가 직무공통인 경우 상위 5%과목의 빈도수
중분류 = '직무공통'
percent_5=int(len(행정안전부[행정안전부['중분류']==중분류].과정명.value_counts())*0.05)
plt.figure(figsize=(30,20))
plt.title('중분류가 {}인 관측치들의 상위5% 과목별 빈도수'.format(중분류))
ax=sns.barplot(y=행정안전부[행정안전부.중분류==중분류].과정명.value_counts()[:percent_5].index,x=행정안전부[행정안전부.중분류==중분류].과정명.value_counts()[:percent_5])
ax.set_xlabel("빈도수")


In [ ]:
#중분류가 사회복지 경우 상위 20개 과목의 빈도수
중분류 = '사회복지'
percent_5=int(len(행정안전부[행정안전부['중분류']==중분류].과정명.value_counts())*0.05)
if percent_5>=5:
  plt.figure(figsize=(20,15))
  plt.title('중분류가 {}인 관측치들의 과목별 빈도수'.format(중분류))
  ax=sns.barplot(y=행정안전부[행정안전부.중분류==중분류].과정명.value_counts()[:percent_5].index,x=행정안전부[행정안전부.중분류==중분류].과정명.value_counts()[:percent_5])
  ax.set_xlabel("빈도수")
else:
  plt.figure(figsize=(20,15))
  plt.title('중분류가 {}인 관측치들의 과목별 빈도수'.format(중분류))
  ax=sns.barplot(y=행정안전부[행정안전부.중분류==중분류].과정명.value_counts()[:20].index,x=행정안전부[행정안전부.중분류==중분류].과정명.value_counts()[:20])
  ax.set_xlabel("빈도수")


In [ ]:
#중분류가 청렴인 경우 상위 20개 과목의 빈도수
중분류 = '청렴'
percent_5=int(len(행정안전부[행정안전부['중분류']==중분류].과정명.value_counts())*0.05)
if percent_5>=5:
  plt.figure(figsize=(20,15))
  plt.title('중분류가 {}인 관측치들의 상위5% 과목별 빈도수'.format(중분류))
  ax=sns.barplot(y=행정안전부[행정안전부.중분류==중분류].과정명.value_counts()[:percent_5].index,x=행정안전부[행정안전부.중분류==중분류].과정명.value_counts()[:percent_5])
  ax.set_xlabel("빈도수")
else:
  plt.figure(figsize=(20,15))
  plt.title('중분류가 {}인 관측치들의 상위 20개 과목별 빈도수'.format(중분류))
  ax=sns.barplot(y=행정안전부[행정안전부.중분류==중분류].과정명.value_counts()[:20].index,x=행정안전부[행정안전부.중분류==중분류].과정명.value_counts()[:20])
  ax.set_xlabel("빈도수")


In [ ]:
#1.1.5
중분류 = '정보화'
percent_5=int(len(행정안전부[행정안전부['중분류']==중분류].과정명.value_counts())*0.05)
if percent_5>=5:
  plt.figure(figsize=(20,15))
  plt.title('중분류가 {}인 관측치들의 상위5% 과목별 빈도수'.format(중분류))
  ax=sns.barplot(y=행정안전부[행정안전부.중분류==중분류].과정명.value_counts()[:percent_5].index,x=행정안전부[행정안전부.중분류==중분류].과정명.value_counts()[:percent_5])
  ax.set_xlabel("빈도수")
else:
  plt.figure(figsize=(20,15))
  plt.title('중분류가 {}인 관측치들의 상위 20개 과목별 빈도수'.format(중분류))
  ax=sns.barplot(y=행정안전부[행정안전부.중분류==중분류].과정명.value_counts()[:20].index,x=행정안전부[행정안전부.중분류==중분류].과정명.value_counts()[:20])
  ax.set_xlabel("빈도수")

In [ ]:
중분류 = '정보화'
행정안전부[행정안전부.중분류==중분류].과정명.value_counts()

In [ ]:
#1.1.6
중분류 = '외국어'
percent_5=int(len(행정안전부[행정안전부['중분류']==중분류].과정명.value_counts())*0.05)
plt.figure(figsize=(20,15))
plt.title('중분류가 {}인 관측치들의 과목별 빈도수'.format(중분류))
ax=sns.barplot(y=행정안전부[행정안전부.중분류==중분류].과정명.value_counts().index,x=행정안전부[행정안전부.중분류==중분류].과정명.value_counts())
ax.set_xlabel("빈도수")

In [ ]:
중분류 = '외국어'
행정안전부[행정안전부.중분류==중분류].과정명.value_counts()

In [ ]:
plt.figure(figsize=(20,15))
plt.title('행정안전부 수료기록 데이터의 소분류별 빈도수')
a=행정안전부.소분류.value_counts()
ax=sns.barplot(x=a.index,y=a)
ax.set_xlabel("빈도수")

In [ ]:
#1.1.7
plt.figure(figsize=(20,15))
plt.title('행정안전부 수료기록 데이터의 소분류별 비율')
행정안전부.groupby('소분류').count()['수료여부'].plot(kind='pie',explode=[0,0,0,0,0.1,0,0,0,0],autopct='%1.1f%%')

In [ ]:
# 1.1.8
소분류 = '인권'
percent_5=int(len(행정안전부[행정안전부['소분류']==소분류].과정명.value_counts())*0.05)
if percent_5>5:
  plt.figure(figsize=(20,15))
  plt.title('소분류가 {}인 관측치들의 상위5% 과목별 빈도수'.format(소분류))
  ax=sns.barplot(y=행정안전부[행정안전부.소분류==소분류].과정명.value_counts()[:percent_5].index,x=행정안전부[행정안전부.소분류==소분류].과정명.value_counts()[:percent_5])
  ax.set_xlabel("빈도수")
else: 
  cut=len(행정안전부[행정안전부.소분류==소분류].과정명.value_counts())
  plt.figure(figsize=(20,15))
  plt.title('소분류가 {}인 관측치들의 과목별 빈도수'.format(소분류))
  ax=sns.barplot(y=행정안전부[행정안전부.소분류==소분류].과정명.value_counts()[:cut].index,x=행정안전부[행정안전부.소분류==소분류].과정명.value_counts()[:cut])
  ax.set_xlabel("빈도수")
  print(len(행정안전부[행정안전부.소분류==소분류].과정명.value_counts()))


In [ ]:
소분류 = '정책'
percent_5=int(len(행정안전부[행정안전부['소분류']==소분류].과정명.value_counts())*0.05)
if percent_5>5:
  plt.figure(figsize=(20,15))
  plt.title('소분류가 {}인 관측치들의 상위5% 과목별 빈도수'.format(소분류))
  ax=sns.barplot(y=행정안전부[행정안전부.소분류==소분류].과정명.value_counts()[:percent_5].index,x=행정안전부[행정안전부.소분류==소분류].과정명.value_counts()[:percent_5])
  ax.set_xlabel("빈도수")
else: 
  cut=len(행정안전부[행정안전부.소분류==소분류].과정명.value_counts())
  plt.figure(figsize=(20,15))
  plt.title('소분류가 {}인 관측치들의 상위 20개 과목별 빈도수'.format(소분류))
  ax=sns.barplot(y=행정안전부[행정안전부.소분류==소분류].과정명.value_counts()[:cut].index,x=행정안전부[행정안전부.소분류==소분류].과정명.value_counts()[:cut])
  ax.set_xlabel("빈도수")
  print(len(행정안전부[행정안전부.소분류==소분류].과정명.value_counts()))


In [ ]:
소분류 = '안보'
percent_5=int(len(행정안전부[행정안전부['소분류']==소분류].과정명.value_counts())*0.05)
plt.figure(figsize=(20,15))
if percent_5>=5:
  plt.title('소분류가 {}인 관측치들의 상위5% 과목별 빈도수'.format(소분류))
  ax=sns.barplot(y=행정안전부[행정안전부.소분류==소분류].과정명.value_counts()[:percent_5].index,x=행정안전부[행정안전부.소분류==소분류].과정명.value_counts()[:percent_5])
  ax.set_xlabel("빈도수")
  print(percent_5)
elif len(행정안전부[행정안전부.소분류==소분류].과정명.value_counts()) <= 10:
  k=len(행정안전부[행정안전부.소분류==소분류].과정명.value_counts())
  plt.title('소분류가 {}인 관측치들의 과목별 빈도수'.format(소분류))
  ax=sns.barplot(y=행정안전부[행정안전부.소분류==소분류].과정명.value_counts()[:k].index,x=행정안전부[행정안전부.소분류==소분류].과정명.value_counts()[:k])
  ax.set_xlabel("빈도수")
  print(len(행정안전부[행정안전부.소분류==소분류].과정명.value_counts()))
else:
  plt.title('소분류가 {}인 관측치들의 상위 20개 과목별 빈도수'.format(소분류))
  ax=sns.barplot(y=행정안전부[행정안전부.소분류==소분류].과정명.value_counts()[:20].index,x=행정안전부[행정안전부.소분류==소분류].과정명.value_counts()[:20])
  ax.set_xlabel("빈도수")
  print(len(행정안전부[행정안전부.소분류==소분류].과정명.value_counts()))

In [ ]:
소분류 = '소양'
percent_5=int(len(행정안전부[행정안전부['소분류']==소분류].과정명.value_counts())*0.05)
plt.figure(figsize=(20,15))
if percent_5>=5:
  plt.title('소분류가 {}인 관측치들의 상위5% 과목별 빈도수'.format(소분류))
  ax=sns.barplot(y=행정안전부[행정안전부.소분류==소분류].과정명.value_counts()[:percent_5].index,x=행정안전부[행정안전부.소분류==소분류].과정명.value_counts()[:percent_5])
  ax.set_xlabel("빈도수")
  print(percent_5)
elif len(행정안전부[행정안전부.소분류==소분류].과정명.value_counts()) <= 10:
  k=len(행정안전부[행정안전부.소분류==소분류].과정명.value_counts())
  plt.title('소분류가 {}인 관측치들의 과목별 빈도수'.format(소분류))
  ax=sns.barplot(y=행정안전부[행정안전부.소분류==소분류].과정명.value_counts()[:k].index,x=행정안전부[행정안전부.소분류==소분류].과정명.value_counts()[:k])
  ax.set_xlabel("빈도수")
  print(len(행정안전부[행정안전부.소분류==소분류].과정명.value_counts()))
else:
  plt.title('소분류가 {}인 관측치들의 상위 20개 과목별 빈도수'.format(소분류))
  ax=sns.barplot(y=행정안전부[행정안전부.소분류==소분류].과정명.value_counts()[:20].index,x=행정안전부[행정안전부.소분류==소분류].과정명.value_counts()[:20])
  ax.set_xlabel("빈도수")
  print(len(행정안전부[행정안전부.소분류==소분류].과정명.value_counts()))

In [ ]:
소분류 = '정보화'
percent_5=int(len(행정안전부[행정안전부['소분류']==소분류].과정명.value_counts())*0.05)
plt.figure(figsize=(20,15))
if percent_5>=5:
  plt.title('소분류가 {}인 관측치들의 상위5% 과목별 빈도수'.format(소분류))
  ax=sns.barplot(y=행정안전부[행정안전부.소분류==소분류].과정명.value_counts()[:percent_5].index,x=행정안전부[행정안전부.소분류==소분류].과정명.value_counts()[:percent_5])
  ax.set_xlabel("빈도수")
  print(percent_5)
elif len(행정안전부[행정안전부.소분류==소분류].과정명.value_counts()) <= 10:
  k=len(행정안전부[행정안전부.소분류==소분류].과정명.value_counts())
  plt.title('소분류가 {}인 관측치들의 과목별 빈도수'.format(소분류))
  ax=sns.barplot(y=행정안전부[행정안전부.소분류==소분류].과정명.value_counts()[:k].index,x=행정안전부[행정안전부.소분류==소분류].과정명.value_counts()[:k])
  ax.set_xlabel("빈도수")
  print(len(행정안전부[행정안전부.소분류==소분류].과정명.value_counts()))
else:
  plt.title('소분류가 {}인 관측치들의 상위 20개 과목별 빈도수'.format(소분류))
  ax=sns.barplot(y=행정안전부[행정안전부.소분류==소분류].과정명.value_counts()[:20].index,x=행정안전부[행정안전부.소분류==소분류].과정명.value_counts()[:20])
  ax.set_xlabel("빈도수")
  print(len(행정안전부[행정안전부.소분류==소분류].과정명.value_counts()))

In [ ]:
소분류 = '성인지'
percent_5=int(len(행정안전부[행정안전부['소분류']==소분류].과정명.value_counts())*0.05)
plt.figure(figsize=(20,15))
if percent_5>=5:
  plt.title('소분류가 {}인 관측치들의 상위5% 과목별 빈도수'.format(소분류))
  ax=sns.barplot(y=행정안전부[행정안전부.소분류==소분류].과정명.value_counts()[:percent_5].index,x=행정안전부[행정안전부.소분류==소분류].과정명.value_counts()[:percent_5])
  ax.set_xlabel("빈도수")
  print(percent_5)
elif len(행정안전부[행정안전부.소분류==소분류].과정명.value_counts()) <= 10:
  k=len(행정안전부[행정안전부.소분류==소분류].과정명.value_counts())
  plt.title('소분류가 {}인 관측치들의 과목별 빈도수'.format(소분류))
  ax=sns.barplot(y=행정안전부[행정안전부.소분류==소분류].과정명.value_counts()[:k].index,x=행정안전부[행정안전부.소분류==소분류].과정명.value_counts()[:k])
  ax.set_xlabel("빈도수")
  print(len(행정안전부[행정안전부.소분류==소분류].과정명.value_counts()))
else:
  plt.title('소분류가 {}인 관측치들의 상위 20개 과목별 빈도수'.format(소분류))
  ax=sns.barplot(y=행정안전부[행정안전부.소분류==소분류].과정명.value_counts()[:20].index,x=행정안전부[행정안전부.소분류==소분류].과정명.value_counts()[:20])
  ax.set_xlabel("빈도수")
  print(len(행정안전부[행정안전부.소분류==소분류].과정명.value_counts()))

In [ ]:
소분류 = '외국어'
percent_5=int(len(행정안전부[행정안전부['소분류']==소분류].과정명.value_counts())*0.05)
plt.figure(figsize=(20,15))
if percent_5>=5:
  plt.title('소분류가 {}인 관측치들의 상위5% 과목별 빈도수'.format(소분류))
  ax=sns.barplot(y=행정안전부[행정안전부.소분류==소분류].과정명.value_counts()[:percent_5].index,x=행정안전부[행정안전부.소분류==소분류].과정명.value_counts()[:percent_5])
  ax.set_xlabel("빈도수")
  print(percent_5)
elif len(행정안전부[행정안전부.소분류==소분류].과정명.value_counts()) <= 10:
  k=len(행정안전부[행정안전부.소분류==소분류].과정명.value_counts())
  plt.title('소분류가 {}인 관측치들의 과목별 빈도수'.format(소분류))
  ax=sns.barplot(y=행정안전부[행정안전부.소분류==소분류].과정명.value_counts()[:k].index,x=행정안전부[행정안전부.소분류==소분류].과정명.value_counts()[:k])
  ax.set_xlabel("빈도수")
  print(len(행정안전부[행정안전부.소분류==소분류].과정명.value_counts()))
else:
  plt.title('소분류가 {}인 관측치들의 상위 20개 과목별 빈도수'.format(소분류))
  ax=sns.barplot(y=행정안전부[행정안전부.소분류==소분류].과정명.value_counts()[:20].index,x=행정안전부[행정안전부.소분류==소분류].과정명.value_counts()[:20])
  ax.set_xlabel("빈도수")
  print(len(행정안전부[행정안전부.소분류==소분류].과정명.value_counts()))

In [ ]:
소분류 = '재난'
percent_5=int(len(행정안전부[행정안전부['소분류']==소분류].과정명.value_counts())*0.05)
plt.figure(figsize=(20,15))
if percent_5>=5:
  plt.title('소분류가 {}인 관측치들의 상위5% 과목별 빈도수'.format(소분류))
  ax=sns.barplot(y=행정안전부[행정안전부.소분류==소분류].과정명.value_counts()[:percent_5].index,x=행정안전부[행정안전부.소분류==소분류].과정명.value_counts()[:percent_5])
  ax.set_xlabel("빈도수")
  print(1,percent_5)
elif len(행정안전부[행정안전부.소분류==소분류].과정명.value_counts()) <=10:
  k=len(행정안전부[행정안전부.소분류==소분류].과정명.value_counts())
  plt.title('소분류가 {}인 관측치들의 과목별 빈도수'.format(소분류))
  ax=sns.barplot(y=행정안전부[행정안전부.소분류==소분류].과정명.value_counts()[:k].index,x=행정안전부[행정안전부.소분류==소분류].과정명.value_counts()[:k])
  ax.set_xlabel("빈도수")
  print(2,len(행정안전부[행정안전부.소분류==소분류].과정명.value_counts()))
else:
  plt.title('소분류가 {}인 관측치들의 상위 20개 과목별 빈도수'.format(소분류))
  ax=sns.barplot(y=행정안전부[행정안전부.소분류==소분류].과정명.value_counts()[:20].index,x=행정안전부[행정안전부.소분류==소분류].과정명.value_counts()[:20])
  ax.set_xlabel("빈도수")
  print(3,len(행정안전부[행정안전부.소분류==소분류].과정명.value_counts()))

## 중분류별,소분류별 수료율 분석(수료율 분석)


### 데이터 전처리

In [ ]:
행정안전부.수료여부.value_counts()

In [ ]:
행정안전부['수료여부']=행정안전부['수료여부'].map({"수료":1,"미수료":0,"취소":2,"진행중":3,"미승인":4})

In [ ]:
행정안전부

In [ ]:
df1=행정안전부.loc[(행정안전부.수료여부 == 1) | (행정안전부.수료여부 == 0),:]
df1

### 중분류별 수료율 분석

In [ ]:
plt.figure(figsize=(15,10))
plt.title("중분류별 수료율")
sns.barplot(x=df1.groupby('중분류')['수료여부'].mean().sort_values(ascending=False).index,y=df1.groupby('중분류')['수료여부'].mean().sort_values(ascending=False).values)
plt.ylabel("수료율")
plt.xticks(rotation= 30)

###소분류별 수료율 분석

In [ ]:
#1.2.1
plt.figure(figsize=(15,10))
plt.title("소분류별 수료율")
sns.barplot(x=df1.groupby('소분류')['수료여부'].mean().sort_values(ascending=False).index,y=df1.groupby('소분류')['수료여부'].mean().sort_values(ascending=False).values)
plt.ylabel("수료율")
plt.xticks(rotation= 30)

### 각각의 소분류에 속한 과목별 수료율
- 소분류로 구분하여 과목별 수료율을 조사했습니다.  
- 수강한 인원이 30명이상인 과목만 대상입니다.

In [ ]:
plt.subplots(figsize=(20,10))

order=df1.groupby('소분류')['수료여부'].mean().sort_values(ascending=False).index
소분류 = order[0]
data=df1[df1['소분류'] == 소분류].groupby('과정명')['수료여부'].mean()
over_30=df1[df1['소분류'] == 소분류].groupby('과정명')['수료여부'].count()>=30
data=data[over_30]
data=data.sort_values(ascending=False)
ex=np.array((data==data.max()).values,dtype=float)
ex[ex == 1]=0.2
plt.title("소분류가 {}인 관측치들의 과목별 수료율".format(소분류))
sns.barplot(y=data.index,x=data)



In [ ]:
plt.subplots(figsize=(20,10))

order=df1.groupby('소분류')['수료여부'].mean().sort_values(ascending=False).index
소분류 = order[1]
data=df1[df1['소분류'] == 소분류].groupby('과정명')['수료여부'].mean()
over_30=df1[df1['소분류'] == 소분류].groupby('과정명')['수료여부'].count()>=30
data=data[over_30]
data=data.sort_values(ascending=False)
ex=np.array((data==data.max()).values,dtype=float)
ex[ex == 1]=0.2
plt.title("소분류가 {}인 관측치들의 과목별 수료율".format(소분류))
sns.barplot(y=data.index,x=data)



In [ ]:
plt.subplots(figsize=(20,10))

order=df1.groupby('소분류')['수료여부'].mean().sort_values(ascending=False).index
소분류 = order[2]
data=df1[df1['소분류'] == 소분류].groupby('과정명')['수료여부'].mean()
over_30=df1[df1['소분류'] == 소분류].groupby('과정명')['수료여부'].count()>=30
data=data[over_30]
data=data.sort_values(ascending=False)
ex=np.array((data==data.max()).values,dtype=float)
ex[ex == 1]=0.2
plt.title("소분류가 {}인 관측치들의 과목별 수료율".format(소분류))
sns.barplot(y=data.index,x=data)



In [ ]:
plt.subplots(figsize=(20,10))

order=df1.groupby('소분류')['수료여부'].mean().sort_values(ascending=False).index
소분류 = order[3]
data=df1[df1['소분류'] == 소분류].groupby('과정명')['수료여부'].mean()
over_30=df1[df1['소분류'] == 소분류].groupby('과정명')['수료여부'].count()>=30
data=data[over_30]
data=data.sort_values(ascending=False)
ex=np.array((data==data.max()).values,dtype=float)
ex[ex == 1]=0.2
plt.title("소분류가 {}인 관측치들의 과목별 수료율".format(소분류))
sns.barplot(y=data.index,x=data)



In [ ]:
#1.2.2
plt.subplots(figsize=(20,10))

order=df1.groupby('소분류')['수료여부'].mean().sort_values(ascending=False).index
소분류 = order[4]
data=df1[df1['소분류'] == 소분류].groupby('과정명')['수료여부'].mean()
over_30=df1[df1['소분류'] == 소분류].groupby('과정명')['수료여부'].count()>=30
data=data[over_30]
data=data.sort_values(ascending=False)
ex=np.array((data==data.max()).values,dtype=float)
ex[ex == 1]=0.2
plt.title("소분류가 {}인 관측치들의 과목별 수료율".format(소분류))
sns.barplot(y=data.index,x=data)



In [ ]:
#소양인 경우 수강자수가 30명 이상인 과목이 너무 많아서 나눠서그림
#수강자수가 300명 이상
plt.subplots(figsize=(20,10))

order=df1.groupby('소분류')['수료여부'].mean().sort_values(ascending=False).index
소분류 = order[5]
data=df1[df1['소분류'] == 소분류].groupby('과정명')['수료여부'].mean()
over_30=df1[df1['소분류'] == 소분류].groupby('과정명')['수료여부'].count()>=300
data=data[over_30]
data=data.sort_values(ascending=False)
ex=np.array((data==data.max()).values,dtype=float)
ex[ex == 1]=0.2
plt.title("소분류가 {}인 관측치들의 과목별 수료율".format(소분류))
sns.barplot(y=data.index,x=data)



In [ ]:
#수강자수 200명 이상 300명 미만
plt.subplots(figsize=(20,10))

order=df1.groupby('소분류')['수료여부'].mean().sort_values(ascending=False).index
소분류 = order[5]
data=df1[df1['소분류'] == 소분류].groupby('과정명')['수료여부'].mean()
over_30=(df1[df1['소분류'] == 소분류].groupby('과정명')['수료여부'].count()<300) & (df1[df1['소분류'] == 소분류].groupby('과정명')['수료여부'].count()>=200)
data=data[over_30]
data=data.sort_values(ascending=False)
ex=np.array((data==data.max()).values,dtype=float)
ex[ex == 1]=0.2
plt.title("소분류가 {}인 관측치들의 과목별 수료율".format(소분류))
sns.barplot(y=data.index,x=data)



In [ ]:
#수강자수 100명 이상 200명 미만
plt.subplots(figsize=(20,10))

order=df1.groupby('소분류')['수료여부'].mean().sort_values(ascending=False).index
소분류 = order[5]
data=df1[df1['소분류'] == 소분류].groupby('과정명')['수료여부'].mean()
over_30=(df1[df1['소분류'] == 소분류].groupby('과정명')['수료여부'].count()<200) & (df1[df1['소분류'] == 소분류].groupby('과정명')['수료여부'].count()>=100)
data=data[over_30]
data=data.sort_values(ascending=False)
ex=np.array((data==data.max()).values,dtype=float)
ex[ex == 1]=0.2
plt.title("소분류가 {}인 관측치들의 과목별 수료율".format(소분류))
sns.barplot(y=data.index,x=data)





In [ ]:
#수강자수 50명 이상 100명 미만
plt.subplots(figsize=(20,10))

order=df1.groupby('소분류')['수료여부'].mean().sort_values(ascending=False).index
소분류 = order[5]
data=df1[df1['소분류'] == 소분류].groupby('과정명')['수료여부'].mean()
over_30=(df1[df1['소분류'] == 소분류].groupby('과정명')['수료여부'].count()<100) & (df1[df1['소분류'] == 소분류].groupby('과정명')['수료여부'].count()>=50)
data=data[over_30]
data=data.sort_values(ascending=False)
ex=np.array((data==data.max()).values,dtype=float)
ex[ex == 1]=0.2
plt.title("소분류가 {}인 관측치들의 과목별 수료율".format(소분류))
sns.barplot(y=data.index,x=data)



In [ ]:
#수강자수 30명 이상 50명 미만
plt.subplots(figsize=(20,10))

order=df1.groupby('소분류')['수료여부'].mean().sort_values(ascending=False).index
소분류 = order[5]
data=df1[df1['소분류'] == 소분류].groupby('과정명')['수료여부'].mean()
over_30=(df1[df1['소분류'] == 소분류].groupby('과정명')['수료여부'].count()<50) & (df1[df1['소분류'] == 소분류].groupby('과정명')['수료여부'].count()>=30)
data=data[over_30]
data=data.sort_values(ascending=False)
ex=np.array((data==data.max()).values,dtype=float)
ex[ex == 1]=0.2
plt.title("소분류가 {}인 관측치들의 과목별 수료율".format(소분류))
sns.barplot(y=data.index,x=data)



In [ ]:
plt.subplots(figsize=(20,10))

order=df1.groupby('소분류')['수료여부'].mean().sort_values(ascending=False).index
소분류 = order[6]
data=df1[df1['소분류'] == 소분류].groupby('과정명')['수료여부'].mean()
over_30=df1[df1['소분류'] == 소분류].groupby('과정명')['수료여부'].count()>=30
data=data[over_30]
data=data.sort_values(ascending=False)
ex=np.array((data==data.max()).values,dtype=float)
ex[ex == 1]=0.2
plt.title("소분류가 {}인 관측치들의 과목별 수료율".format(소분류))
sns.barplot(y=data.index,x=data)



In [ ]:
#정책인 경우 수강자수가 30명 이상인 과목이 너무 많아서 나눠서 그림
#수강자수 300명 이상 
plt.subplots(figsize=(20,10))

order=df1.groupby('소분류')['수료여부'].mean().sort_values(ascending=False).index
소분류 = order[7]
data=df1[df1['소분류'] == 소분류].groupby('과정명')['수료여부'].mean()
over_30=(df1[df1['소분류'] == 소분류].groupby('과정명')['수료여부'].count()>=300)
data=data[over_30]
data=data.sort_values(ascending=False)
ex=np.array((data==data.max()).values,dtype=float)
ex[ex == 1]=0.2
plt.title("소분류가 {}인 관측치들의 과목별 수료율".format(소분류))
sns.barplot(y=data.index,x=data)



In [ ]:
#수강자수 200명 이상 300명 미만
plt.subplots(figsize=(20,10))

order=df1.groupby('소분류')['수료여부'].mean().sort_values(ascending=False).index
소분류 = order[7]
data=df1[df1['소분류'] == 소분류].groupby('과정명')['수료여부'].mean()
over_30=(df1[df1['소분류'] == 소분류].groupby('과정명')['수료여부'].count()<300) & (df1[df1['소분류'] == 소분류].groupby('과정명')['수료여부'].count()>=200)
data=data[over_30]
data=data.sort_values(ascending=False)
ex=np.array((data==data.max()).values,dtype=float)
ex[ex == 1]=0.2
plt.title("소분류가 {}인 관측치들의 과목별 수료율".format(소분류))
sns.barplot(y=data.index,x=data)



In [ ]:
#수강자수 100명 이상 200명 미만
plt.subplots(figsize=(20,10))

order=df1.groupby('소분류')['수료여부'].mean().sort_values(ascending=False).index
소분류 = order[7]
data=df1[df1['소분류'] == 소분류].groupby('과정명')['수료여부'].mean()
over_30=(df1[df1['소분류'] == 소분류].groupby('과정명')['수료여부'].count()<200) & (df1[df1['소분류'] == 소분류].groupby('과정명')['수료여부'].count()>=100)
data=data[over_30]
data=data.sort_values(ascending=False)
ex=np.array((data==data.max()).values,dtype=float)
ex[ex == 1]=0.2
plt.title("소분류가 {}인 관측치들의 과목별 수료율".format(소분류))
sns.barplot(y=data.index,x=data)




In [ ]:
#수강자수 50명 이상 100명 미만
plt.subplots(figsize=(20,10))

order=df1.groupby('소분류')['수료여부'].mean().sort_values(ascending=False).index
소분류 = order[7]
data=df1[df1['소분류'] == 소분류].groupby('과정명')['수료여부'].mean()
over_30=(df1[df1['소분류'] == 소분류].groupby('과정명')['수료여부'].count()<100) & (df1[df1['소분류'] == 소분류].groupby('과정명')['수료여부'].count()>=50)
data=data[over_30]
data=data.sort_values(ascending=False)
ex=np.array((data==data.max()).values,dtype=float)
ex[ex == 1]=0.2
plt.title("소분류가 {}인 관측치들의 과목별 수료율".format(소분류))
sns.barplot(y=data.index,x=data)




In [ ]:
#수강자수 30명 이상 50명 미만
plt.subplots(figsize=(20,10))

order=df1.groupby('소분류')['수료여부'].mean().sort_values(ascending=False).index
소분류 = order[7]
data=df1[df1['소분류'] == 소분류].groupby('과정명')['수료여부'].mean()
over_30=(df1[df1['소분류'] == 소분류].groupby('과정명')['수료여부'].count()<50) & (df1[df1['소분류'] == 소분류].groupby('과정명')['수료여부'].count()>=30)
data=data[over_30]
data=data.sort_values(ascending=False)
ex=np.array((data==data.max()).values,dtype=float)
ex[ex == 1]=0.2
plt.title("소분류가 {}인 관측치들의 과목별 수료율".format(소분류))
sns.barplot(y=data.index,x=data)




In [ ]:
#1.2.3
plt.subplots(figsize=(20,10))

order=df1.groupby('소분류')['수료여부'].mean().sort_values(ascending=False).index
소분류 = order[8]
data=df1[df1['소분류'] == 소분류].groupby('과정명')['수료여부'].mean()
over_30=df1[df1['소분류'] == 소분류].groupby('과정명')['수료여부'].count()>=30
data=data[over_30]
data=data.sort_values(ascending=False)
ex=np.array((data==data.max()).values,dtype=float)
ex[ex == 1]=0.2
plt.title("소분류가 {}인 관측치들의 과목별 수료율".format(소분류))
sns.barplot(y=data.index,x=data)



# 3.  3. 수료자 기준 성인지(남‧녀) 통계 분석
  - 전과정 수료자 성비 분석 ( 1. 과정별 남‧녀 비율 비교 )
  - 과정별 수료자 성비 분석 ( 1. 과정별 남‧녀 비율 비교 )

##전라남도 수료기록 분석

### 데이터 전처리

In [ ]:
전라남도 = dataset1.copy()
전라남도['수료여부']=전라남도['수료여부'].map({"수료":1,"미수료":0,"취소":2,"진행중":3,"미승인":4})

In [ ]:
#수료여부가 널값인 관측치들은 제거
전라남도=전라남도.drop(index=전라남도[pd.isna(전라남도.수료여부)].index.values)
전라남도.reset_index(drop=True)
전라남도=전라남도.astype({'수료여부':int})

In [ ]:
전라남도['성별']=전라남도['성별'].map({'남성':1,'여성':0})

In [ ]:
전라남도=전라남도[(전라남도.수료여부 == 1) | (전라남도.수료여부 == 0)]
전라남도 = 전라남도.reset_index(drop=True)

###전과정 수료자 성비 분석 ( 1. 과정별 남‧녀 비율 비교 )

In [ ]:
d=전라남도[['성별','수료여부']].groupby('성별')['수료여부'].mean().sort_values(ascending=False)
map={0:'여성',1:'남성'}
plt.figure(figsize=(10,7))
x_name = [map[i] for i in d.index]
ax=sns.barplot(x=x_name,y=d)
ax.set_title('성별 수료율')
ax.set_ylabel('수료율')
ax.set_xlabel('성별')
print('남자인 경우 수료율 : {:.2f}'.format(d[1]))
print('여자인 경우 수료율 : {:.2f}'.format(d[0]))

###과정별 수료자 성비 분석 ( 1. 과정별 남‧녀 비율 비교 )

In [ ]:
bool_ind=(전라남도.groupby('과정명').count()['성명']<30)
under_30=전라남도.groupby('과정명').count()['성명'][bool_ind].index.values
print('30명 미만이 수강하는 과목',under_30)
df=전라남도.copy()
df=df.set_index('과정명')
print('\n수강생이 30명 미만인 과정 삭제전',df.shape)
df=df.drop(index=under_30)
print('수강생이 30명 미만인 과정 삭제후',df.shape)

In [ ]:
df=df.reset_index()
df

In [ ]:
data=df.groupby(['과정명','성별'])['수료여부'].mean().to_frame().수료여부
over_30_과정명=data.to_frame().reset_index().과정명.unique()
diff=[]
for i in range(0,len(data),2):
  diff.append(data.values[i]-data.values[i+1])
len(diff)

In [ ]:
#-인 경우 남자의 수료율이 더 많은 과목
#+인 경우 여자의 수료율이 더 많은 과목
diff_df=pd.DataFrame({'과정명':over_30_과정명,'차이':diff})
diff_df['절댓값 차이']=abs(diff_df['차이'])
diff_df=diff_df.sort_values('절댓값 차이',ascending=False)

In [ ]:
diff_df

In [ ]:
plt.figure(figsize=(15,10))
plt.title('성비차이가 많이나는 과정 상위 10개')
sns.barplot(data=diff_df.iloc[:10,:],x='절댓값 차이',y='과정명')

In [ ]:
diff_df[diff_df['차이']<0]

In [ ]:
plt.subplots(1,2,figsize=(20,10))
plt.subplot(1,2,1)
plt.title('남자가 더 많이 듣는 과정 상위 10개')
sns.barplot(data=diff_df[diff_df['차이']<0].iloc[:10,:],x='절댓값 차이',y='과정명')


data2=diff_df[diff_df['차이']<0].iloc[-10:,:].sort_values('절댓값 차이')
plt.subplot(1,2,2)
plt.title('남자가 더 많이 듣는 과정 하위 10개')
sns.barplot(data=data2,x='절댓값 차이',y='과정명')

In [ ]:
#여자가 많이듣는 과목수는 남자가 많이듣는 과목수보다 적음
#남자인 관측치가 더 많기 때문이라고 보임
diff_df[diff_df['차이']>0]

In [ ]:
print('데이터에서 \n남자인 관측치의 수 : {}\n여자인 관측치의 수 : {}'.format(sum(df.성별==1),len(df)-sum(df.성별==1)))
plt.figure(figsize=(15,10))
plt.title('전라남도 수료기록 데이터의 남녀 비율')
plt.pie(x=전라남도.groupby('성별').count()['수료여부'],explode=[0,0.1],autopct='%1.2f%%',shadow=True,labels=['여성','남성'])

In [ ]:
plt.figure(figsize=(20,10))
plt.subplot(1,2,1)
plt.title('여자가 더 많이 듣는 과정 상위 10개')
sns.barplot(data=diff_df[diff_df['차이']>0].iloc[:10,:],x='절댓값 차이',y='과정명')

data2=diff_df[diff_df['차이']>0].iloc[-10:,:].sort_values('절댓값 차이')
plt.subplot(1,2,2)
plt.title('여자가 더 많이 듣는 과정 하위 10개')
sns.barplot(data=data2,x='절댓값 차이',y='과정명')

## 행정안전부 수료기록 분석

In [ ]:
df1['성별']=df1.성별.map({'남성':1,'여성':0})
df1

In [ ]:
under_30_과정명=df1.groupby('과정명').count()['수료여부'][df1.groupby('과정명').count()['수료여부']<30].index
print('30명 미만이 수강하는 과목',under_30_과정명)
df4=df1.copy()
df4=df4.set_index('과정명')
print('\n수강생이 30명 미만인 과정 삭제 후',df4.shape)
df4=df4.drop(index=under_30_과정명)
print('수강생이 30명 미만인 과정 삭제 후',df4.shape)

In [ ]:
df4=df4.reset_index()
df4

In [ ]:
data=df4.groupby(['과정명','성별'])['수료여부'].mean().to_frame().수료여부
over_30_과정명=data.to_frame().reset_index().과정명.unique()
diff=[]
for i in range(0,len(data),2):
  diff.append(data.values[i]-data.values[i+1])
len(diff)

In [ ]:
data=df4.groupby(['과정명','성별'])['수료여부'].mean().to_frame().수료여부
data

In [ ]:
#-인 경우 남자의 수료율이 더 많은 과목
#+인 경우 여자의 수료율이 더 많은 과목
diff_df=pd.DataFrame({'과정명':over_30_과정명,'차이':diff})
diff_df['절댓값 차이']=abs(diff_df['차이'])
diff_df=diff_df.sort_values('절댓값 차이',ascending=False)

In [ ]:
diff_df

In [ ]:
plt.figure(figsize=(15,10))
plt.title('성비차이가 많이나는 과정 상위 10개')
sns.barplot(data=diff_df.iloc[:10,:],x='절댓값 차이',y='과정명')

In [ ]:
#남성이 더 많이듣는 과정의 수
diff_df[diff_df['차이']<0].shape

In [ ]:
plt.subplots(1,2,figsize=(20,10))
plt.subplot(1,2,1)
plt.title('남자가 더 많이 듣는 과정 상위 10개')
sns.barplot(data=diff_df[diff_df['차이']<0].iloc[:10,:],x='절댓값 차이',y='과정명')


data2=diff_df[diff_df['차이']<0].iloc[-10:,:].sort_values('절댓값 차이')
plt.subplot(1,2,2)
plt.title('남자가 더 많이 듣는 과정 하위 10개')
sns.barplot(data=data2,x='절댓값 차이',y='과정명')

In [ ]:
#여자가 많이듣는 과목수는 남자가 많이듣는 과목수보다 적음
#남자인 관측치가 더 많기 때문이라고 보임
diff_df[diff_df['차이']>0]

In [ ]:
행정안전부.groupby('성별').count()['수료여부']

In [ ]:
print('데이터에서 \n남자인 관측치의 수 : {}\n여자인 관측치의 수 : {}'.format(sum(df4.성별==1),len(df4)-sum(df4.성별==1)))
plt.figure(figsize=(15,10))
plt.title('행정안전부 수료기록 데이터의 남녀 비율')
plt.pie(x=행정안전부.groupby('성별').count()['수료여부'],explode=[0,0.1],autopct='%1.2f%%',shadow=True,labels=['남성','여성'])

In [ ]:
plt.figure(figsize=(20,10))
plt.subplot(1,2,1)
plt.title('여자가 더 많이 듣는 과정 상위 10개')
sns.barplot(data=diff_df[diff_df['차이']>0].iloc[:10,:],x='절댓값 차이',y='과정명')

data2=diff_df[diff_df['차이']>0].iloc[-10:,:].sort_values('절댓값 차이')
plt.subplot(1,2,2)
plt.title('여자가 더 많이 듣는 과정 하위 10개')
sns.barplot(data=data2,x='절댓값 차이',y='과정명')